# Домашнее задание №1

В этом домашнем задании вам предлагается обучить модель регрессии для предсказания стоимости автомобилей, а также реализовать веб-сервис для применения построенной модели на новых данных.

> Оценка за домашку = min(ваш балл, 10)


Задания, <font color='#9933CC'>выделенные фиолетовым</font>, требуют от вас написания кода.

А вопросы, <font color='#FF6600'>выделенные оранжевым</font>, текстового ответа.

Оцениваются как код, так и ответы на вопросы. Если нет одного и/или другого, то часть баллов за соответствующее задание без колебаний снимается.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

random.seed(42)
np.random.seed(42)

Ниже ответьте на вопрос <font color='#FF6600'>
"Для чего фиксируем сиды в домашках?"</font>

In [ ]:
print("Чтобы все получили результаты, которые можно объективно (без влияния рандома) сравнить друг с другом")

# Часть 1 (2.5 балла + 0.75) | EDA

## Простейший EDA и обработка признаков (1.5 балла + 0.25)

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/hse-mlds/ml/main/hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/hse-mlds/ml/main/hometasks/HT1/cars_test.csv')

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

(0.15 балла) <font color='#9933CC'>Отобразите 10 **случайных** строк тренировочного датасета</font>

In [ ]:
df_train.sample(10)

(0.1 балла) <font color='#9933CC'>Отобразите первые 5 и последние 5 объектов тестового датасета</font>

In [ ]:
df_test.head(5)

In [ ]:
df_test.tail(5)

(0.1 балла) <font color='#9933CC'>Посчитайте основные статистики как по числовым, так и по категориальным столбцам для трейна и теста</font>. 
> Подсказка: ``.describe()`` с нужным(и) аргументом(-ами)



In [ ]:
df_train.dtypes

In [ ]:
df_train.describe(include = ['object'])

In [ ]:
df_train.describe(include = ['int64','float64']).round(2)

In [ ]:
df_test.describe(include = ['object'])

In [ ]:
df_test.describe(include = ['object'])

In [ ]:
df_test.describe(include = ['int64','float64']).round(2)

(0.15 балла) <font color='#9933CC'>Посмотрите, есть ли в датасете пропуски.</font>

<font color='#FF6600'>Если есть, то в каких колонках?</font>

In [ ]:
#seats, mileage, engine, max_power, torque

(0.1 балла) <font color='#9933CC'>Посмотрите, есть ли в трейне объекты с одинаковым признаковым описанием</font> (целевую переменную следует исключить). Если есть, то сколько?

In [ ]:
df_train_copy = df_train.drop('selling_price', axis = 1).copy()

(0.15 балла) <font color='#9933CC'>Отобразите такие объекты</font>

In [ ]:
df_train[df_train_copy.duplicated()]

(0.15 балла) <font color='#9933CC'>Удалите повторяющиеся строки</font>. Если при одинаковом признаковом описании цены на автомобили отличаются, то оставьте первую строку по этому автомобилю

In [ ]:
df_train_copy = df_train_copy.drop_duplicates(keep='first')

In [ ]:
df_train = df_train.iloc[df_train_copy.index]

In [ ]:
assert df_train.shape == (5840, 13)

(0.1 балла) Чтоб все было по красоте, <font color='#9933CC'>обновите индексы строк таким образом, чтобы они шли от 0 без пропусков</font>

In [ ]:
df_train = df_train.reset_index(drop=True)

Вы могли заметить, что с признаками ``mileage, engine, max_power и torque`` всё не слава богу. Давайте починим.

(0.25 балла) Задача такая: 
<font color='#9933CC'>
* убрать единицы измерения для признаков ``mileage, engine, max_power``. 
* кастануть эти столбцы столбцы к ``float``.
* удалить столбец ``torque``
</font>

> Все действия нужно производить над обоими датасетами

---
**Доп (0.25 балла):**
* <font color='#9933CC'>Вместо удаления признак `torque` разделите на два: собственно `torque` и `max_torque_rpm`.</font> Учтите единицы измерения. Они разные ☹

In [ ]:
df_train.mileage = df_train.mileage.str.extract('(\d+)', expand=False).astype(float)
df_train.engine = df_train.engine.str.extract('(\d+)', expand=False).astype(float)
df_train.max_power = df_train.max_power.str.extract('(\d+)', expand=False).astype(float)
df_train = df_train.drop('torque', axis = 1)

In [ ]:
df_test.mileage = df_test.mileage.str.extract('(\d+)', expand=False).astype(float)
df_test.engine = df_test.engine.str.extract('(\d+)', expand=False).astype(float)
df_test.max_power = df_test.max_power.str.extract('(\d+)', expand=False).astype(float)
df_test = df_test.drop('torque', axis = 1)

(0.15 балла) <font color='#9933CC'>Заполните пропуски в столбцах медианами. Убедитесь, что после заполнения пропусков не осталось</font>

> Обратите внимание, что, по уму, нужно посчитать медиану по трейну и этим средним заполнять пропуски в тесте. Так же делаем, если, например, стандартизируем признаки.



In [ ]:
for col in ['seats', 'mileage', 'engine', 'max_power']:
    df_train[col] = df_train[col].fillna(df_train[col].median())
    df_test[col] = df_test[col].fillna(df_train[col].median())

In [ ]:
df_train.describe(include = ['object'])

In [ ]:
df_train.describe(include = ['int64','float64']).round(2)

In [ ]:
df_test.describe(include = ['object'])

In [ ]:
df_test.describe(include = ['int64','float64']).round(2)


(0.1 балла) Теперь, когда не осталось пропусков, можно <font color='#9933CC'>преобразовать столбцы к более подходящим типам (``engnine`` и ``seats`` к int)
</font>

Ниже ответьте, <font color='#FF6600'>почему (хоть мы этого и не делаем) ``seats``, возможно, лучше сделать переменной категориальной, а не целочисленной.</font>

In [ ]:
print("Очевидно, что количество сидений ограничено и +- заранее известно, не считая кейсов с условным автобусом")

In [ ]:
df_train['engine'] = df_train['engine'].astype(int)
df_train['seats'] = df_train['seats'].astype(int)

df_test['engine'] = df_test['engine'].astype(int)
df_test['seats'] = df_test['seats'].astype(int)

## Визуализации (1 балл + 0.5)

Визуализировать нам надо не так уж и много. Во-первых, хотелось бы, в принципе, увидеть как распределены значения признаков. Также хотелось бы понять, насколько признаки скоррелированы между собой и с целевой переменной. А ещё неплохо бы посмотреть, не оказалось ли так, что тестовые данные распределены иначе, чем трейн.

Этим всем и предлагаем вам заняться.

(0.25 балла) <font color='#9933CC'>Посторойте попарные распределения всех числовых признаков для трейна.</font>

> ``sns.paiplot()`` позволяет сделать это в одну строчку

In [ ]:
sns.pairplot(df_train)

In [ ]:
sns.heatmap(df_train.select_dtypes('number').corr(), cmap="Blues", annot=True)

Этот график не такой информативный, как можно было бы построить, беря признаки по отдельности. Но он позволяет сделать некоторые (возможно, далеко идущие) выводы. Впрочем, со многими из них (если не со всеми) вы бы справились умозрительно. Однако всегда приятно делать выводы, основываясь на данных.



* <font color='#FF6600'> Что можно сказать о связи предикторов с целевой переменной?
* А о корреляциях признаков? </font>

In [ ]:
# Сильно с целевой переменно коррелирует max_power, это видно и на графике, и в матрице корреляций
# меньше - год и объем двигателя
# остальные не существенно

#Между собой сильно коррелируют объем двигателя и его мощность, что закономерно)
#Также, как ни странно, есть сильная корреляция между объемом двигателя и кол-вом сидений в автомобиле

(0.25 балла) <font color='#9933CC'>Постройте pairplot по тестовым данным</font> и ответьте на вопрос <font color='#FF6600'>"Похожими ли оказались совокупности при разделении на трейн и тест?"</font>

In [ ]:
sns.pairplot(df_test)

In [ ]:
sns.heatmap(df_test.select_dtypes('number').corr(), cmap="Blues", annot=True)

(0.25 балла) <font color='#9933CC'>Для трейна давайте построим тепловую карту (heatmap из библиотеки seaborn) попарных корреляций числовых колонок</font>

In [ ]:
# сделал выше

Ответьте, пожалуйста, на вопросы: 
* <font color='#FF6600'>Какие 2 признака наименее скоррелированы между собой?</font>
* <font color='#FF6600'>Между какими наблюдается довольно сильная положительная линейная зависимость?</font>
* <font color='#FF6600'>Правильно ли, опираясь на данные, утверждать, что чем меньше год, тем, скорее всего, больше километров проехала машина к дате продажи.</font>

In [ ]:
#1) engine и year
#2) max_power и engine
#3) Да, можно, есть достаточно большая отрицательная корреляция (-0.37) на трейне и (-0.47) на тесте

(0.25 балла) <font color='#9933CC'> Отобразите диаграммe рассеяния для наиболее скореллированной пары **признаков** (на трейне) </font>

In [ ]:
df_train.plot(x='max_power', y='engine', style='o')

### Дополнительные визуализации (бонус 0.5 балла)

Если вам кажется, что мы не попросили вас нарисовать какие-то очень важные зависимости, нарисуйте их и поясните.

In [ ]:
df_train.plot(x='max_power', y='selling_price', style='o')

In [ ]:
#Корелляция между ценой и макс мощностью двигателя, тк она тоже существенная

In [ ]:
df_train.plot(x='engine', y='seats', style='o') #Так как признак числа сидений 
#- по большей части категориальный, на графике особо не видно зависимости, однако корреляция есть

# Часть 2 (1.25 балла) | Модель только на вещественных признаках

(0.05 балла) <font color='#9933CC'>В переменные ``y_train`` и ``y_test`` запишите значения целевых переменных. Столбцы ``selling_price`` из датафреймов необходимо удалить. Категориальные (все, кроме ``seats``) столбцы тоже.</font>

In [ ]:
df_train

In [ ]:
y_train = df_train['selling_price']
X_train = df_train.drop(['name','selling_price', 'fuel','seller_type', 'transmission', 'owner'], axis = 1)

In [ ]:
assert X_train.shape == (5840, 6)

In [ ]:
y_test = df_test['selling_price']
X_test = df_test.drop(['name','selling_price', 'fuel','seller_type', 'transmission', 'owner'], axis = 1)

In [ ]:
assert X_test.shape == (1000, 6)

(0.2 балла) <font color='#9933CC'>Обучите классическую линейную регрессию с дефолтными параметрами. Посчтитайте $R^2$ и MSE для трейна и для теста</font>

**Замечание:** $R^2$ и MSE для трейна и для теста выводите везде, где требуется обучать модели, даже если в явном виде этого не просят. Иначе непонятно, как понять, насколько успешны наши эксперименты.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error as MSE

model = LinearRegression()

model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(f'R2: {r2_score(y_test,y_preds)}')
print(f'MSE: {MSE(y_test,y_preds)}')

Запомните правило:

> Использую линейную модель -- стандартизирую фичи

(0.25 балла) <font color='#9933CC'>Воспользуемся им</font>


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(f'R2: {r2_score(y_test,y_preds)}')
print(f'MSE: {MSE(y_test,y_preds)}')

Не очень результативно.

Зато уже сейчас можем интерпретировать модель. <font color='#FF6600'>"Какой признак оказался наиболее информативным в предсказании цены?"</font>

In [ ]:
model.coef_

In [ ]:
df_train

In [ ]:
#как и ожидалось, объем двигателя, у которого наибольшая корреляция с целевой переменной

(0.25 балла) <font color='#9933CC'>Теперь попробуем Lasso-регрессию.</font> Здесь и дальше обучайте модели на нормализованных признаках

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso()
model.fit(X_train, y_train)
y_preds = model.predict(X_test)

print(f'R2: {r2_score(y_test,y_preds)}')
print(f'MSE: {MSE(y_test,y_preds)}')

<font color='#FF6600'>Занулила ли L1-регуляризация с параметрами по умолчанию какие-нибудь веса? Почему же?</font>

In [ ]:
model.coef_

In [ ]:
#нет, не занулила, веса большие, а параметр 1 по дефолту

(0.25 балла) <font color='#9933CC'>Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для Lasso-регрессии</font>

Вам пригодится класс [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
from sklearn.model_selection import GridSearchCV

alpha_range = list(range(1, 100))
param_grid = dict(alpha = alpha_range)

model = Lasso()  

grid = GridSearchCV(model, param_grid, cv = 10, scoring='r2', return_train_score=True)

grid_search=grid.fit(X_train, y_train)

In [ ]:
y_preds = grid_search.best_estimator_.predict(X_test)

print(f'R2: {r2_score(y_test,y_preds)}')
print(f'MSE: {MSE(y_test,y_preds)}')

<font color='#FF6600'>Сколько грид-сёрчу пришлось обучать моделей?</font>

In [ ]:
len(grid_search.cv_results_['mean_fit_time']) # 99

<font color='#FF6600'>Какой коэффициент регуляризации у лучшей из перебранных моделей? Занулились ли какие-нибудь из весов при такой регуляризации?</font>

In [ ]:
grid_search.best_estimator_ # как не странно, максимальный их доступных, значит скорее всего дальше будет лучше

In [ ]:
grid_search.best_estimator_.coef_

In [ ]:
#не занулились

(0.25 балла) <font color='#9933CC'>Перебором по сетке (c 10-ю фолдами) подберите оптимальные параметры для [ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html)-регрессии</font>

In [ ]:
from sklearn.linear_model import ElasticNet

model = ElasticNet()

parametersGrid = {"max_iter": [1000, 2000, 5000],
                      "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                      "l1_ratio": np.arange(0.0, 1.0, 0.1)}


grid = GridSearchCV(model, parametersGrid, cv = 10, scoring='r2', return_train_score=True)

grid_search=grid.fit(X_train, y_train)

In [ ]:
y_preds = grid_search.best_estimator_.predict(X_test)

print(f'R2: {r2_score(y_test,y_preds)}')
print(f'MSE: {MSE(y_test,y_preds)}')

<font color='#FF6600'>Какие гиперпараметры соответствуют лучшей из перебранных моделей?</font>

In [ ]:
grid_search.best_estimator_

Предлагаем вам оставить вам попытки заметно улучшить качество модели регуляризацией и перейти к следующей части задания.

# Часть 3 (0.5 балла) | Добавляем категориальные фичи



(0 баллов) <font color='#9933CC'>Из ``df_train`` удалите столбцы с целевой переменной и названием автомобиля.</font>

In [ ]:
y_train_cat = df_train['selling_price']
X_train_cat = df_train.drop(['name','selling_price'], axis = 1)

In [ ]:
y_test_cat = df_test['selling_price']
X_test_cat = df_test.drop(['name','selling_price'], axis = 1)

In [ ]:
assert X_train_cat.shape == (5840, 10)

In [ ]:
X_train_cat.describe(include='object')

(0.5 балла) <font color='#9933CC'>Закодируйте категориалльные фичи и ``seats`` методом OneHot-кодирования</font>

> Обратите внимание, что во избежание мультиколлинеарности следует избавиться от одного из полученных столбцов при кодировании каждого признака методом OneHot. 

In [ ]:
from sklearn.preprocessing import OneHotEncoder # или можно использовать get_dummies из библиотеки pandas

for col in ['fuel', 'seller_type', 'transmission', 'owner', 'seats']:
    encoder = OneHotEncoder()
    
    one_hot = encoder.fit(X_train_cat[col].to_numpy().reshape(-1,1))
    cols = one_hot.get_feature_names_out()[:-1]
    encoded = encoder.transform(X_train_cat[col].to_numpy().reshape(-1,1))
    encoded = pd.DataFrame(encoded.toarray()[:,:-1], index = X_train_cat.index, columns = cols)
    X_train_cat = X_train_cat.drop(col,axis = 1)
    X_train_cat = pd.concat([X_train_cat, encoded], axis='columns')
    
    encoded = encoder.transform(X_test_cat[col].to_numpy().reshape(-1,1))
    cols = one_hot.get_feature_names_out()[:-1]
    encoded = pd.DataFrame(encoded.toarray()[:,:-1], index = X_test_cat.index, columns = cols)
    X_test_cat = X_test_cat.drop(col,axis = 1)
    X_test_cat = pd.concat([X_test_cat, encoded], axis='columns')

(0.25 балла) <font color='#9933CC'>Переберите параметр регуляризации `alpha` для гребневой (ridge) регрессии с помощью класса `GridSearchCV`.</font> В качестве параметров при объявлении GridSearchCV кроме модели укажите метрику качества $R^2$. Кроссвалидируйтесь по 10-ти фолдам.

<font color='#FF6600'>Удалось ли улучшить качество предсказаний?</font>

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

alpha_range = list(range(1, 100))
param_grid = dict(alpha = alpha_range)

model = Ridge()  

grid = GridSearchCV(model, param_grid, cv = 10, scoring='r2', return_train_score=True)

grid_search=grid.fit(X_train_cat, y_train_cat)

In [ ]:
grid_search.best_estimator_

In [ ]:
y_preds = grid_search.best_estimator_.predict(X_test_cat)

print(f'R2: {r2_score(y_test_cat,y_preds)}')
print(f'MSE: {MSE(y_test_cat,y_preds)}')

In [ ]:
#удалось

# Часть 4 - бонусная (1.5 балла) | Feature Engineering 

В этой части домашнего задания вам предлагается проявить свою креативность для улучшения прогноза модели. Любые другие модели, кроме различных форм линейной (или полиномиальной) регресси, использовать запрещается. А значит, придется работать с признаками

**Что можно попробовать сделать?** (каждый пункт по 0.4 балла, но не больше 2-х баллов в сумме)

1.   *Сгенерировать новые признаки на основе уже существующих:*
    * посчитать произведения // частные признаков (кажется, что посчитать число "лошадей" на литр объема может быть полезно);
    * имеет смысл обратить внимание на визуализации в части с EDA (к примеру, зависимость цены от года выглядит квадратичной, а не линейной; значит, квадрат года нам, скорее всего, принесет больше пользы)

2.   *Добыть новые признаки:*
    * имеем название автомобиля, которое никак не используем (можно спарсить инфу о классе автомобиля или каких-то специфических опциях)
    * можно добавить пороговые признаки вроде "владелец третий или больше" и объединить признаки в некоторые осмысленные правила, например "первый или второй владелец и продавец официальный дилер" (подбирать пороги удобно по диаграммам рассеяния)

3.   *Поработать с уже имеющимися:*
    * далеко не факт, что заполнить пропуск медианой было лучшей идеей (как минимум, можно добавить dummy-столбец для модели, сигнализирующий, что раньше на месте медианы был пропуск -- там где он был, конечно); попробуйте другие способы филлинга;
    * мы не анализировали, есть ли в данных выбросы => никак выбросы не обрабатывали; наиболее простым и, тем не менее, довольно полезным вариантом нахождения выбросов могут послужить boxplot'ы для каждого столбца; что делать с выбросами думайте сами :) -- вариантов довольно много
    * мы толком не смотрели на таргет сам по себе; в нем тоже могут быть неожиданности -- стоит хотя бы проверить
    * можно заметить, что некоторые признаки распределены совсем не нормально; возможно их стоит отлогарифмировать

И так далее...

Feel Free to Try!

In [ ]:
df_train

In [1]:
def feature_engineering(df_train, df_test):
    df_train['year_sqr'] = df_train['year'] * df_train['year']
    df_test['year_sqr'] = df_test['year'] * df_test['year']
    
    df_train['hp_by_volume'] = df_train['max_power'] / df_train['engine']
    df_test['hp_by_volume'] = df_test['max_power'] / df_test['engine']
    
    return df_train, df_test

In [3]:
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error as MSE
import pandas as pd

df_train = pd.read_csv('https://raw.githubusercontent.com/hse-mlds/ml/main/hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/hse-mlds/ml/main/hometasks/HT1/cars_test.csv')

df_train_copy = df_train.drop('selling_price', axis = 1).copy()
df_train_copy = df_train_copy.drop_duplicates(keep='first')
df_train = df_train.iloc[df_train_copy.index]

df_train = df_train.reset_index(drop=True)

df_train.mileage = df_train.mileage.str.extract('(\d+)', expand=False).astype(float)
df_train.engine = df_train.engine.str.extract('(\d+)', expand=False).astype(float)
df_train.max_power = df_train.max_power.str.extract('(\d+)', expand=False).astype(float)
df_train = df_train.drop('torque', axis = 1)

df_test.mileage = df_test.mileage.str.extract('(\d+)', expand=False).astype(float)
df_test.engine = df_test.engine.str.extract('(\d+)', expand=False).astype(float)
df_test.max_power = df_test.max_power.str.extract('(\d+)', expand=False).astype(float)
df_test = df_test.drop('torque', axis = 1)

for col in ['seats', 'mileage', 'engine', 'max_power']:
    df_train[col] = df_train[col].fillna(df_train[col].median())
    df_test[col] = df_test[col].fillna(df_train[col].median())

df_train['engine'] = df_train['engine'].astype(int)
df_train['seats'] = df_train['seats'].astype(int)

df_test['engine'] = df_test['engine'].astype(int)
df_test['seats'] = df_test['seats'].astype(int)

y_train_cat = df_train['selling_price']
X_train_cat = df_train.drop(['name','selling_price'], axis = 1)

y_test_cat = df_test['selling_price']
X_test_cat = df_test.drop(['name','selling_price'], axis = 1)


encoder = OneHotEncoder(drop = 'first')
one_hot = encoder.fit(X_train_cat[['fuel', 'seller_type', 'transmission', 'owner', 'seats']])

cols = one_hot.get_feature_names_out()
encoded = encoder.transform(X_train_cat[['fuel', 'seller_type', 'transmission', 'owner', 'seats']])
encoded = pd.DataFrame(encoded.toarray(), index = X_train_cat.index, columns = cols)
X_train_cat = X_train_cat.drop(['fuel', 'seller_type', 'transmission', 'owner', 'seats'],axis = 1)
X_train_cat = pd.concat([X_train_cat, encoded], axis='columns')

encoded = encoder.transform(X_test_cat[['fuel', 'seller_type', 'transmission', 'owner', 'seats']])
encoded = pd.DataFrame(encoded.toarray(), index = X_test_cat.index, columns = cols)
X_test_cat = X_test_cat.drop(['fuel', 'seller_type', 'transmission', 'owner', 'seats'],axis = 1)
X_test_cat = pd.concat([X_test_cat, encoded], axis='columns')


X_train_cat,X_test_cat = feature_engineering(X_train_cat,X_test_cat) 
    
scaler = StandardScaler()

X_train_cat = pd.DataFrame(scaler.fit_transform(X_train_cat))
X_test_cat = pd.DataFrame(scaler.transform(X_test_cat))

In [ ]:
alpha_range = list(range(1, 100))
param_grid = dict(alpha = alpha_range)

model = Ridge()  

alpha_range = list(range(1, 100))
param_grid = dict(alpha = alpha_range)

grid = GridSearchCV(model, param_grid, cv = 10, scoring='r2', return_train_score=True)

grid_search=grid.fit(X_train_cat, y_train_cat)

y_preds = grid_search.best_estimator_.predict(X_test_cat)

print(f'R2: {r2_score(y_test_cat,y_preds)}')
print(f'MSE: {MSE(y_test_cat,y_preds)}')

In [4]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(logging_level = 'Silent')

model.fit(X_train_cat, y_train_cat)

y_preds = model.predict(X_test_cat)

print(f'R2: {r2_score(y_test_cat,y_preds)}')
print(f'MSE: {MSE(y_test_cat,y_preds)}')

R2: 0.9723086223227903
MSE: 15917794633.82917


In [ ]:
#catboost из коробки как обычно очень хорош...

In [ ]:
import joblib
joblib.dump(encoder, 'encoder.pkl')
joblib.dump(scaler, 'scaler.pkl')
joblib.dump(model, 'model.pkl')

# Часть Бизнесовая (0.5 балла)

Заказчик просил вас посчитать кастомную метрику -- среди всех предсказанных цен на авто посчитать долю предиктов, отличающихся от реальных цен на эти авто не более чем на 10% (в одну или другую сторону)

<font color='#9933CC'>Сделайте это самостоятельно для лучшей из своих моделей</font>

In [8]:
def business_metrics(y_true, y_pred):
    import numpy as np
    
    metric = (np.abs(y_true - y_pred) / y_true > 0.1).sum() / y_pred.shape[0]
    return metric

business_metrics(y_test_cat,y_preds)

0.56

# Часть 5 (2.5 балла) | Реализация сервиса на FastAPI

Cделайте с помощью FastAPI сервис, который с точки зрения пользователя реализует две функции:

1. на вход в формате json подаются признаки одного объекта, на выходе сервис выдает предсказанную стоимость машины
2. на вход подается csv-файл с признаками тестовых объектов, на выходе получаем файл с +1 столбцом - предсказаниями на этих объектах

С точки зрения реализации это означает следующее:
- средствами pydantic должен быть описан класс базового объекта
- класс с коллецией объектов
- метод post, который получает на вход один объект описанного класса
- метод post, который получает на вход коллекцию объектов описанного класса

Шаблон для сервисной части дан ниже. Код необходимо дополнить и оформить в виде отдельного .py-файла.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

app = FastAPI()


class Item(BaseModel):
    name: str
    year: int
    selling_price: int
    km_driven: int
    fuel: str
    seller_type: str
    transmission: str
    owner: str
    mileage: str 
    engine: str
    max_power: str
    torque: str
    seats: float


class Items(BaseModel):
    objects: List[Item]


@app.post("/predict_item")
def predict_item(item: Item) -> float:
    return ...


@app.post("/predict_items")
def predict_items(items: List[Item]) -> List[float]:
    return ...

Протестируйте сервис на корректность работы и приложите скриншоты (см. ниже).

# Часть 6 (1 балл) | Оформление результатов

**Результаты вашей работы** необходимо разместить в своем Гитхабе. Под результатами понимаем следующее:
* ``.ipynb``-ноутбук со всеми проведёнными вами экспериментами (output'ы ячеек, разумеется, сохранить)
* ``.py``-файл с реализацией сервиса
* ``.pickle``-файл с сохранёнными весами модели, коэффициентами скейлинга и прочими числовыми значениями, которые могут понадобиться для инференса
* ``.md``-файл с выводами про проделанной вами работе:
    * что было сделано
    * с какими результатами
    * что дало наибольший буст в качестве
    * что сделать не вышло и почему (это нормально, даже хорошо😀)

**За что могут быть сняты баллы в этом пункте:**
* за отсутствие ``.pickle``-файла с весами использованной модели
* за недостаточную аналитику в ``.md``-файле
* за оформление и логику кода (в определённом смысле это тоже элемент оформления решения)

**Как будет выглядет проверка всего домашнего задания?**
1. Ассистент проходит по ссылке на (**открытый**) репозиторий из Энитаска
2. Смотрит ``readme.md``:
    * пожалуйста, приложите в него же скрины работы вашего сервиса -- собирать ваши проекты довольно времязатратно, но хочется убедиться, что всё работает
    * можете прислать screencast в ТГ
3. Просматривает ноутбук с pure-DS частью
4. Заглядывает в код сервиса
5. Хвалит

# Часть Благодарственная

Надеемся, вы честно проделали все пункты, а не просто пролистали досюда. Потому что здесь награда за старания. Пожалуйста, не стоит награждать себя до того, как закончите работать над домашкой.

https://drive.google.com/file/d/1LT06nyg9QgVi3r-D5BY-_mQ9TcJQFxJN/view?usp=share_link